In [1]:
# Позволяет использовать измененные модули без перезагрузки ядра
%load_ext autoreload
%autoreload 2

import tensorflow_probability
from Mixture import mixture
import numpy as np

2024-02-11 19:04:25.331976: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Generate mixture, check EMs on valid initial parameters, compare results 

In [108]:

# Create mixture model
m = mixture(
    num_comps=3,
    distrib=tensorflow_probability.distributions.Normal
    )
np.random.seed()
rseed = np.random.randint(1_000)
m.initialize_probs_mus_sigmas(random_seed=rseed)
print(f"radnom seed equal to {rseed}")
# Generate data for testing
m.generate_samples(1_000, random_seed=57)
another_data = m.construct_tpf_mixture().sample(1_000).numpy()
test_data1 = m.samples
test_data2 = another_data

## additional stuff; output appearancece
__round = lambda numb, dig=4: round(numb, dig)
Round = lambda lis, dig=4: list(map(lambda numb: round(numb, dig),lis))
custom_print = lambda text, *args, func=Round: print(
    text, 
    f"Orig - {func(args[0])}",
    f"Iter - {func(args[1])}",
    f"Adap - {func(args[2])}",
    sep='\n\t')

# EM comparison
pit, mit, sit, llhit = m.EM_iterative(test_data1, 10)
pad, mad, sad, llhad = m.EM_adaptive(test_data1, 0.01)

custom_print("Components probabilities:", m.probs, pit, pad)
custom_print("Mathematical expectations:", m.mus, mit, mad)
custom_print("Standard deviations:", m.sigmas, sit, sad)
custom_print("Log-likelihoods:", m.log_likelihood(test_data1), llhit, llhad, 
             func=__round)


radnom seed equal to 237
Components probabilities:
	Orig - [0.9389, 0.0581, 0.003]
	Iter - [0.9387, 0.0575, 0.0038]
	Adap - [0.9218, 0.0566, 0.0216]
Mathematical expectations:
	Orig - [16.3757, 3.2124, 19.5925]
	Iter - [16.356, 2.3665, 20.0636]
	Adap - [16.2431, 2.2922, 21.4602]
Standard deviations:
	Orig - [3.7022, 3.3992, 2.8342]
	Iter - [3.773, 2.5775, 2.1429]
	Adap - [3.7418, 2.5193, 1.1095]
Log-likelihoods:
	Orig - -2927.3407
	Iter - -2925.0522
	Adap - -2923.8535


In [3]:
m.plot().show()

In [4]:

from plotly import express
express.line(m.samples)

#### Checking sieving EM

In [5]:
np.random.seed()
rseed = np.random.randint(1_000)
m = mixture(
    num_comps=3,
    distrib=tensorflow_probability.distributions.Normal,
    random_seed=rseed,
    rand_initialize=True
    )

print(f"random seed equal to {rseed}")
# Generate data for testing
m.generate_samples(1_000, random_seed=rseed)

from debug_prints import round_sort
# print("Original probs:",round_sort(m.probs, 4),'\n')
res = m.EM_sieving(
    m.samples, 
    iter_initial=8, 
    num_candid=50,
    num_best_candid=10,
    accur_final=0.001,
    random_seed=rseed)

print("Original mixture:",
      round_sort(m.probs, 4), 
      m.mus, 
      m.sigmas, 
      sep='\n')
print("Sieved EM result:", *res, sep='\n')
orig_mat = np.array([m.probs, m.mus, m.sigmas])
pred_mat = np.array([res[0], res[1], res[2]])
print("\nNorma for not sorded matrixes: ", np.linalg.norm(pred_mat - orig_mat))
print("\nDistance between originals and predictions - ", 
      round(np.linalg.norm(np.sort(m.probs)-np.sort(res[0])), 4))

random seed equal to 770
[ 2.01301689e+01  3.99085653e+00  7.29914564e+00  4.52384404e+00
  6.89872495e+00  2.11307251e+01  1.18854139e+01  3.71198017e+00
  6.26893152e+00  1.80367448e+01  2.01757922e+01  1.30204227e+00
  1.84901060e+00  3.56783088e+00  5.57788433e+00  6.82038592e+00
  2.14562903e+01  6.65917326e+00  7.70338556e+00  7.09405276e+00
  1.87787063e+01  1.45780199e+01  5.03935007e+00  7.29471918e+00
  2.71450790e+00  3.71232465e+00  1.79204552e+01  1.72574922e+01
  8.54585231e+00  8.42062058e-01  5.01679520e+00  2.10704734e+00
  1.46735597e+01  1.53437062e+01  4.18019949e+00  1.45029639e+01
  1.87807528e+01  5.76091029e+00  4.93716076e+00  1.58872418e+01
  1.48121040e+01  1.70978303e+01  2.34972871e+00 -2.53651205e+00
  4.93208917e+00  1.20343845e+01  6.24277083e+00  6.90545517e+00
  2.96413774e+00  1.07478396e+01  1.73146518e+01  1.47123563e+01
  8.88655732e+00  2.80624444e+00  5.26854516e-01  5.54170801e-01
  1.74415467e+01  7.87245770e+00  1.80246766e-04  6.51611971e+00


In [6]:
m.plot()

In [7]:
print(m.probs, m.mus, m.sigmas, sep='\n')

[0.24088054 0.05289033 0.70622912]
[16.34209001 13.67393503  5.51765523]
[3.19513773 1.41759485 3.2970299 ]


In [8]:
from numpy.lib.stride_tricks import sliding_window_view

x = np.arange(100_000)
w = sliding_window_view(x,4)
print(w)

[[    0     1     2     3]
 [    1     2     3     4]
 [    2     3     4     5]
 ...
 [99994 99995 99996 99997]
 [99995 99996 99997 99998]
 [99996 99997 99998 99999]]


## Testing dynamix mixture class

Suppose we have constant mathematical expectation and standard deviation
parameters. However probabilities are dynamic:
$$
p_0(x) = \frac{1}{2} \sin(x) + \frac{1}{2} \quad
p_1(x) = \frac{3}{10} \left(-\frac{1}{2} \sin(x) + \frac{1}{2}\right) \quad
p_2(x) = \frac{7}{10} \left(-\frac{1}{2} \sin(x) + \frac{1}{2}\right) \quad
$$

In [2]:
from Mixture import DynamicMixture

dm=DynamicMixture(
    num_comps=3,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=np.linspace(0,np.pi,50),
    window_shape=(10, 1))

In [71]:

SIN = lambda a=0, b=0, n=1, scale=1: lambda x: np.sin(n*x + a) * scale + b 
COS = lambda a=0, b=0, n=1, scale=1: lambda x: np.cos(n*x + a) * scale + b

p0 = SIN(b=1/2, scale=1/2)
p1 = SIN(b=1* 0.3/2, scale=-1* 0.3/2) 
p2 = SIN(b=1* 0.7/2, scale=-1* 0.7/2) 

quad_sinx = lambda x: np.power(np.sin(x),2)
ident = lambda x: x
div_cosNx = lambda n: lambda x: np.cos(n*x) / 2 + 0.49
constant = lambda const: lambda x : np.float64(const)

probs_beh = (quad_sinx, constant(1/2-0.49),  div_cosNx(2))
mus_beh = (constant(1.2), constant(2.5),  constant(0))
sigmas_beh = (constant(0.2), constant(0.5), constant(0.8))


In [73]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(rows=1, cols=1, subplot_titles=("Plot 1", "Plot 2", "Plot 3"))
x = np.linspace(-np.pi,np.pi, 100)
# Add traces
for i,p in enumerate((p0,p1,p2)):
    fig.add_trace(go.Scatter(x=x, y=p(x), mode='lines', name=f'p{i}'), row=1, col=1)

# Update layout
fig.update_layout(title_text="Multiple Subplots")
fig.show()


In [74]:
dm.generate_samples(1, (probs_beh, mus_beh, sigmas_beh))

/home/oplora/Documents/VMK/Dissertation/Magfield/Algorithms.py:18: RuntimeWarning:

invalid value encountered in log



In [3]:
from Converter import converter

DATA = converter("dropna_january_march.csv", dir='Data')
DATA.change_time_format()
data_clean = DATA.dropna_january_march

In [49]:

dm=DynamicMixture(
    num_comps=2,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=np.linspace(0,np.pi,50),
    window_shape=(4500, 50))
dm2=DynamicMixture(
    num_comps=2,
    distrib=tensorflow_probability.distributions.Normal,
    time_span=np.linspace(0,np.pi,50),
    window_shape=(4500, 50))
bx = data_clean['BX'].values

In [ ]:

dm.predict(data=bx[:5000],random_seed=12)

In [51]:
dm2.predic_light(data=bx[:5000],random_seed=12)

Bad selection in ITER. Restarting the iteration  3
(array([0.85764854, 0.14235146]), array([-0.17813903, -3.64054648]), array([2.22594379, 0.76122883]), -10243.833441908564)


  0%|          | 0/10 [00:00<?, ?it/s]

(array([0.86313377, 0.13686623]), array([-0.1378646 , -3.63845912]), array([2.21871334, 0.75428517]), -10238.160727962848)
(array([0.87357995, 0.12642005]), array([-0.10609418, -3.61175514]), array([2.21734047, 0.75339425]), -10230.348760877685)
(array([0.88173069, 0.11826931]), array([-0.05680312, -3.51722166]), array([2.22348419, 0.70045405]), -10206.86940410364)
(array([0.88756588, 0.11243412]), array([-0.03201159, -3.52724931]), array([2.20784412, 0.70468239]), -10189.126987928325)
(array([0.89211149, 0.10788851]), array([-0.00770654, -3.54071699]), array([2.19539908, 0.70851195]), -10174.680732082245)
(array([0.90190404, 0.09809596]), array([ 0.00978606, -3.443357  ]), array([2.17963545, 0.64944665]), -10106.60242862585)
(array([0.9058926, 0.0941074]), array([ 0.05887537, -3.36590942]), array([2.14535345, 0.56194765]), -10017.291685438013)
(array([0.91468118, 0.08531882]), array([ 0.0844303 , -3.33814764]), array([2.14036253, 0.54959271]), -9998.522757859282)
(array([0.91985719, 0

'finished'

In [41]:
import numpy as np

# Your original dictionary
original_dict = {
    'list1': [np.array([1.1, 2.2, 3.3]), np.array([4.4, 5.5, 6.6]), np.array([7.7, 8.8, 9.9])],
    'list2': [np.array([0.1, 0.2, 0.3]), np.array([0.4, 0.5, 0.6]), np.array([0.7, 0.8, 0.9])]
}

# Reshaping the values of the dictionary
reshaped_dict = {}
for key, value in original_dict.items():
    reshaped_arrays = []
    for i in range(len(value[0])):
        reshaped_arrays.append(np.array([arr[i] for arr in value]))
    reshaped_dict[key] = reshaped_arrays
reshaped_dict.clear()
# Print the reshaped dictionary
print(reshaped_dict)

{}


In [52]:
dm2.parameters

{'probs': [array([0.85764854, 0.14235146]),
  array([0.85764854, 0.14235146]),
  array([0.86313377, 0.13686623]),
  array([0.87357995, 0.12642005]),
  array([0.88173069, 0.11826931]),
  array([0.88756588, 0.11243412]),
  array([0.89211149, 0.10788851]),
  array([0.90190404, 0.09809596]),
  array([0.9058926, 0.0941074]),
  array([0.91468118, 0.08531882]),
  array([0.91985719, 0.08014281]),
  array([0.92791011, 0.07208989])],
 'mus': [array([-0.17813903, -3.64054648]),
  array([-0.17813903, -3.64054648]),
  array([-0.1378646 , -3.63845912]),
  array([-0.10609418, -3.61175514]),
  array([-0.05680312, -3.51722166]),
  array([-0.03201159, -3.52724931]),
  array([-0.00770654, -3.54071699]),
  array([ 0.00978606, -3.443357  ]),
  array([ 0.05887537, -3.36590942]),
  array([ 0.0844303 , -3.33814764]),
  array([ 0.12154133, -3.32914857]),
  array([ 0.14142273, -3.30332695])],
 'sigmas': [array([2.22594379, 0.76122883]),
  array([2.22594379, 0.76122883]),
  array([2.21871334, 0.75428517]),
  arr

In [38]:
dm2.show_parameters()

[0, 1]

In [65]:
del dm2.parameters  

here


In [23]:
erer = dict(a=[1,2,4,5], b=[34324,65643])
type(list(erer.items())[0])

tuple

In [45]:
for a,b in zip(['z','x','c'], [88, 77, 66,89]):
    print(a,b)

z 88
x 77
c 66


In [46]:
*a, h = (1,2,3,4)

In [48]:
print(h)

4


In [86]:
list(erer.values())

[[1, 2, 4, 5], [34324, 65643]]

In [94]:
from plotly.express import line
line(dm.llh)

AttributeError: 'DynamicMixture' object has no attribute 'llh'